In [50]:
import requests
import json 
from time import sleep, time
from pandas import json_normalize
import pandas as pd

with open("token.txt", "r") as file:
    token = file.read().strip('\n')

base_url = "https://api.clashroyale.com/v1"

headers = {"Authorization": f"Bearer {token}"}

# Archivo donde guardaremos los resultados
output_file = "clanes_data.json"

# Lista de tags de clanes (ejemplo)
clan_tags = ["#L9CRC0PG", "#GG0C0VR0", "#P00CUJ2R", "#LC0GJGRV", "#YVQV2VVY", "#9JUP2U", "#RPQ2QGYC"] 

def get_clan_data(clan_tag):
    endpoint = f"/clans/{clan_tag.replace('#', '%23')}"  # Codificar el # como %23
    url = base_url + endpoint
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        sleep(0.1)  # Pequeña pausa para evitar rate limiting
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos del clan {clan_tag}: {e}")
        return None

def fetch_all_clans_data(tags, output_file):
    start_time = time()  # Iniciar medición de tiempo
    all_data = {}
    total_clans = len(tags)
    
    for i, tag in enumerate(tags, 1):
        print(f"Obteniendo datos del clan {tag} ({i}/{total_clans})...")
        clan_data = get_clan_data(tag)
        
        if clan_data is not None:
            all_data[tag] = clan_data
    
    # Guardar todos los datos
    with open(output_file, "w") as f:
        json.dump(all_data, f, indent=4)
    
    # Calcular tiempo transcurrido
    elapsed_time = time() - start_time
    mins, secs = divmod(elapsed_time, 60)
    
    print(f"\nProceso completado en {int(mins)} minutos y {secs:.2f} segundos")
    print(f"Datos de {len(all_data)}/{total_clans} clanes guardados en {output_file}")
    
    return all_data

In [51]:
fetch_all_clans_data(clan_tags, output_file)

Obteniendo datos del clan #L9CRC0PG (1/7)...
Obteniendo datos del clan #GG0C0VR0 (2/7)...
Obteniendo datos del clan #P00CUJ2R (3/7)...
Obteniendo datos del clan #LC0GJGRV (4/7)...
Obteniendo datos del clan #YVQV2VVY (5/7)...
Obteniendo datos del clan #9JUP2U (6/7)...
Obteniendo datos del clan #RPQ2QGYC (7/7)...

Proceso completado en 0 minutos y 1.73 segundos
Datos de 7/7 clanes guardados en clanes_data.json


{'#L9CRC0PG': {'tag': '#L9CRC0PG',
  'name': 'Biscy Saucers',
  'type': 'inviteOnly',
  'description': '1# Clan in Canada 🇨🇦 Better than the Rest! Very Active and Supportive! Path of Legends Focused 🏆 Full? Join biscy saucers 2!',
  'badgeId': 16000076,
  'clanScore': 90000,
  'clanWarTrophies': 1650,
  'location': {'id': 57000047,
   'name': 'Canada',
   'isCountry': True,
   'countryCode': 'CA'},
  'requiredTrophies': 9000,
  'donationsPerWeek': 2000,
  'clanChestStatus': 'inactive',
  'clanChestLevel': 1,
  'clanChestMaxLevel': 0,
  'members': 50,
  'memberList': [{'tag': '#JJU8V9YR',
    'name': 'BigDanny224',
    'role': 'coLeader',
    'lastSeen': '20250529T050542.000Z',
    'expLevel': 69,
    'trophies': 9000,
    'arena': {'id': 54000020, 'name': 'Valkalla'},
    'clanRank': 1,
    'previousClanRank': 1,
    'donations': 116,
    'donationsReceived': 0,
    'clanChestPoints': 0},
   {'tag': '#C22U88U2',
    'name': 'Bo$$',
    'role': 'coLeader',
    'lastSeen': '20250529T0008

In [85]:

# 1. Cargar el archivo correctamente
with open('clanes_data.json', 'r') as f:
    clan_data = json.load(f)  # clan_data es ahora un diccionario

# 2. Convertir a DataFrame (estructura deseada)
df = (
    pd.DataFrame.from_dict(clan_data, orient='index')
    .reset_index()
    .rename(columns={'index': 'tag'})
)

# 3. Si hay datos anidados (ej: "location": {"country": "Chile"}):
df_final = pd.json_normalize(
    df.to_dict('memberList'),  # Datos a normalizar
    sep='_',
    max_level= 3                # Separador para campos anidados
)

# Mostrar resultado
df_final.head(5)

/tmp/ipykernel_5964/4285346352.py:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df.to_dict('memberList'),  # Datos a normalizar


ValueError: orient 'memberlist' not understood

In [81]:
df_final = json_normalize(df_transformed.to_dict('records'))

In [82]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tag_clan              7 non-null      object
 1   tag                   7 non-null      object
 2   name                  7 non-null      object
 3   type                  7 non-null      object
 4   description           7 non-null      object
 5   badgeId               7 non-null      int64 
 6   clanScore             7 non-null      int64 
 7   clanWarTrophies       7 non-null      int64 
 8   requiredTrophies      7 non-null      int64 
 9   donationsPerWeek      7 non-null      int64 
 10  clanChestStatus       7 non-null      object
 11  clanChestLevel        7 non-null      int64 
 12  clanChestMaxLevel     7 non-null      int64 
 13  members               7 non-null      int64 
 14  memberList            7 non-null      object
 15  location.id           7 non-null      int64 

In [70]:
df = pd.read_json('clanes_data.json')

In [63]:
#transformando el dataframe
df_transformed = df.reset_index().rename(columns={'index': 'tag_clan'})
#normalizando columnas
df_final = json_normalize(df_transformed.to_dict('records'))

In [80]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tag_clan              7 non-null      object
 1   tag                   7 non-null      object
 2   name                  7 non-null      object
 3   type                  7 non-null      object
 4   description           7 non-null      object
 5   badgeId               7 non-null      int64 
 6   clanScore             7 non-null      int64 
 7   clanWarTrophies       7 non-null      int64 
 8   requiredTrophies      7 non-null      int64 
 9   donationsPerWeek      7 non-null      int64 
 10  clanChestStatus       7 non-null      object
 11  clanChestLevel        7 non-null      int64 
 12  clanChestMaxLevel     7 non-null      int64 
 13  members               7 non-null      int64 
 14  memberList            7 non-null      object
 15  location.id           7 non-null      int64 

In [65]:
df_final.sample(5)

,tag_clan,tag,name,type,description,badgeId,clanScore,clanWarTrophies,requiredTrophies,donationsPerWeek,clanChestStatus,clanChestLevel,clanChestMaxLevel,members,memberList,location.id,location.name,location.isCountry,location.countryCode
1,#GG0C0VR0,#GG0C0VR0,Jynxzi Verse,inviteOnly,,16000008,90000,180,9000,3944,inactive,1,0,50,"[{'tag': '#YJPLQRR09', 'name': 'Lil Log', 'rol...",57000006,International,False,NaN
3,#LC0GJGRV,#LC0GJGRV,los mercenarios,inviteOnly,"Se regalan 3 pass a los 3 top guerra. 2,200 pt...",16000077,90000,4922,9000,3815,inactive,1,0,50,"[{'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', ...",57000169,Nicaragua,True,NI
5,#9JUP2U,#9JUP2U,HOLLIGANS,open,clan de chill pero con ganas de competir / obl...,16000029,79510,1833,1300,1802,inactive,1,0,49,"[{'tag': '#29809G2LL', 'name': 'fernand0', 'ro...",57000153,Mexico,True,MX
4,#YVQV2VVY,#YVQV2VVY,BAD BOYS,inviteOnly,♨️CLAN COMPETITIVO♨️ATACAR EN GUERRA 4/4 ATAQU...,16000009,90000,3693,9000,4148,inactive,1,0,50,"[{'tag': '#JQ82089QR', 'name': 'xoguh12', 'rol...",57000153,Mexico,True,MX
6,#RPQ2QGYC,#RPQ2QGYC,ONE PIECE,open,ONE PIECE İZLEYİN İZLETTİRİN!!! Klan savaşları...,16000024,76289,510,6500,2720,inactive,1,0,41,"[{'tag': '#Y08V0CPLR', 'name': 'allah', 'role'...",57000239,Turkey,True,TR


In [66]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tag_clan              7 non-null      object
 1   tag                   7 non-null      object
 2   name                  7 non-null      object
 3   type                  7 non-null      object
 4   description           7 non-null      object
 5   badgeId               7 non-null      int64 
 6   clanScore             7 non-null      int64 
 7   clanWarTrophies       7 non-null      int64 
 8   requiredTrophies      7 non-null      int64 
 9   donationsPerWeek      7 non-null      int64 
 10  clanChestStatus       7 non-null      object
 11  clanChestLevel        7 non-null      int64 
 12  clanChestMaxLevel     7 non-null      int64 
 13  members               7 non-null      int64 
 14  memberList            7 non-null      object
 15  location.id           7 non-null      int64 